# L2 FWI Example
We will solve this problem using the following steps:
1. Read the true and starting models from binary files, coarsen, and visualize
2. Build up a small local compute cluster and load required packages 
3. Build F, the nonlinear distributed block operator for seismic modeling
4. Use F and the true model to create a synthetic data-set
5. Build the gradient and cost functions
6. Perform the FWI using optim.jl

## Add required packages
Uncomment the line below if you need to add these packages to your environment

In [ ]:
# ]add Optim LineSearches

In [ ]:
using PyPlot, Distributed

## 1. Read true and starting models from binary files

In [ ]:
v = read!("../20_marmousi_model_setup/marmousi_vp_20m_176x851.bin", Array{Float32}(undef, 176,851));
vₒ= read!("../20_marmousi_model_setup/marmousi_vp_smooth_20m_176x851.bin", Array{Float32}(undef, 176,851));

#### Subsample models
We will resample from the 20m grid to a 40m grid

In [ ]:
v = v[1:3:end,1:3:end]
vₒ = vₒ[1:3:end,1:3:end];

In [ ]:
dz,dx = 60.0,60.0
nz,nx = size(v)
@show dz,dx
@show nz,nx;

#### Visualize

In [ ]:
figure(figsize=(16,8))
subplot(121); imshow(v,aspect="auto",cmap="jet"); 
colorbar(orientation="horizontal"); clim(1500,5500); title("True velocity")
subplot(122); imshow(vₒ,aspect="auto",cmap="jet");
colorbar(orientation="horizontal"); clim(1500,5500); title("Starting velocity");

## 2. Build up a small local compute cluster and load required packages 

In [ ]:
nthread = Sys.CPU_THREADS
ENV["OMP_DISPLAY_ENV"] = "true"
ENV["OMP_PROC_BIND"] = "spread"
ENV["GOMP_CPU_AFFINITY"] = "0-$(nthread)"
addprocs(2)
@show workers()
@spawnat workers()[1] ENV["GOMP_CPU_AFFINITY"] = "0-$(div(nthread,2)-1)";
@spawnat workers()[2] ENV["GOMP_CPU_AFFINITY"] = "$(div(nthread,2))-$(nthread-1)";

#### Setup OMP environment variable for the cluster
Note we need to do this because we are using 2 workers on the same physical node

In [ ]:
@everywhere using Distributed, DistributedArrays, DistributedJets, Jets, JetPack, WaveFD, JetPackWaveFD, LinearAlgebra, LineSearches, Optim, Random

## 3. Build F, the forward modeling operator

8 shot locations (only using 8 because we are using a single machine to run the FWI)

In [ ]:
sx = collect(range(0,stop=(nx-1)*dx,length=8))
nshots = length(sx)
@show nshots;
@show sx;

#### Note on scratch space for temporary files
When dealing with serialized nonlinear wavefields as in this example, we need to specify the location where scratch files will be written.

You may need to change this to point to a temporary directory available on your system.

In [ ]:
@everywhere scratch = "/mnt/scratch"
@assert isdir(scratch)

In [ ]:
@everywhere function makeF(i,sx)
    nz,nx,dz,dx = 88,426,40.0,40.0    
    JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        nthreads = div(Sys.CPU_THREADS,2),
        ntrec = 541,
        dtrec = 0.012,
        dtmod = 0.004,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=2.0),
        sx = sx[i],
        sz = dz,
        rx = dx*[0:1:nx-1;],
        rz = 2*dz*ones(length(0:1:nx-1)),
        comptype = UInt32,
        srcfieldfile = joinpath(scratch, "field-$i-$(randstring()).bin"),
        reportinterval=1000)
end

In [ ]:
F = @blockop DArray(I->[makeF(i,sx) for i in I[1], j in I[2]], (nshots,1))

## 4. Use F and the true model to create a synthetic data-set

In [ ]:
# This may take awhile if running on a single node
@time begin
    d = F*v;
end

#### Plot shot gathers from the in-memory distributed array

In [ ]:
ishots = [1, 4, 8, 16]
dmax = maximum(abs, extrema(d))
figure(figsize=(14,5))
for (iplot,ishot) in enumerate(ishots)
    subplot(1,4,iplot);
    imshow(getblock(d,ishot)[:,:,1],cmap="gray",aspect="auto",clim=0.1 .* [-dmax,+dmax]);
    title("shot $(ishot)");
end
tight_layout()

## 5. Build the gradient and cost functions

In [ ]:
#build water bottom mask for gradient
wb_mask = ones(Float32,size(v))
wb_mask[v.==1500.0].=0.0;
imshow(wb_mask,aspect="auto");colorbar();

In [ ]:
function gradient!(G,F,v,dobs,p,wb_mask)
    J = jacobian(F,v)
    s = srcillum(J)
    s .= ((s ./ maximum(s)).^2) .+ 1e-8
    R = JopDiagonal((1 ./ s).^2)
    G .= R' ∘ J' * (dobs - F*v)
    G .*= wb_mask #mute water column
    if p.gscale == 0.0
       p.gscale = 10 ./ maximum(G) #make a scalar from first gradient (then apply to all future gradients)
    end
    G .*= p.gscale
    close(F) #delete local files
end
mutable struct FwiPar
   gscale
end
p = FwiPar(0.0)
g!(G,x) = gradient!(G,F,x,d,p,wb_mask)

In [ ]:
grad = zeros(Float32,size(v))
g!(grad,vₒ)

In [ ]:
figure(figsize=(10,5))
imshow(grad,aspect="auto",cmap="seismic");colorbar();clim(-10,10);title("gradient");

In [ ]:
diffnorm(x, y) = sqrt(mapreduce(i->(x[i] - y[i])^2, +, 1:length(x)))
function cost(x,F,d)
    dm = F*x
    phi = diffnorm(d, dm)
    return phi
end
f(x) = cost(x,F,d)

In [ ]:
@info "initial cost $(f(vₒ))";

## 6. Do the FWI using optim.jl

In [ ]:
solver = LBFGS(m = 20, alphaguess = LineSearches.InitialQuadratic(), linesearch = LineSearches.MoreThuente());

In [ ]:
function mycallback(state::OptimizationState)
    @info "iter=$(state.iteration), cost=$(state.value), |grad|=$(state.g_norm)"
    false
end
mycallback(trace::OptimizationTrace) = mycallback(trace[end]);

In [ ]:
p = FwiPar(0.0)
redirect_stdout(open("/dev/null","w")) # otherwise the models are printed to stdout at every iteration
result = optimize(f, g!, vₒ, solver,
    Optim.Options(
        iterations = 100,
        show_trace = true,
        store_trace = true,
        show_every = 1,
        extended_trace = true,
        allow_f_increases = false,
        callback = mycallback))

In [ ]:
vfwi = Optim.minimizer(result) # optimal solution
ϕ = Optim.f_trace(result)   # cost vs iteration
m = Optim.x_trace(result);   # model vs iteration

In [ ]:
rmprocs(workers())

In [ ]:
figure(figsize=(20,5))
subplot(121);imshow(v,aspect="auto",cmap="jet");colorbar();clim(1500,4500);title("true velocity")
subplot(122);imshow(vfwi,aspect="auto",cmap="jet");colorbar();clim(1500,4500);title("fwi velocity");

In [ ]:
plot(ϕ)